In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool

# ==== 1. 读取数据 ====
df = pd.read_excel("dataset/rating_predict.xlsx")


target = "box_office_category"

candidate_features = [
   '时长','score',  '豆瓣评价人数', '获奖', '提名', '国民总收入亿元',
       '人均国内总产值元', '第三产业增加值亿元', '国内生产总值亿元', '第三产业所占比重国内生产总值100', '第三产业贡献率',
       '全体居民消费水平元', '银幕数量', '影院数量', '演员百度指数', '搜索指数移动日均值', '咨询指数日均值', '疫情影响',
       'others_东亚', 'others_中国', 'others_北美', 'others_南美', 'others_南非',
       'others_欧洲', 'others_法国', 'others_西亚', 'others_五一档', 'others_国庆档',
       'others_普通档', 'others_暑期档', 'others_贺岁档', 'others_剧情', 'others_动作',
       'others_动画', 'others_喜剧', 'others_奇幻', 'others_恐怖', 'others_悬疑',
       'others_战争', 'others_爱情', 'others_科幻', 'others_纪录片', 'others_舞台艺术']

features = [c for c in candidate_features if c in df.columns]
df = df.dropna(subset=[target])
X = df[features].copy()
y = df[target].astype(str)  # 分类标签

# ==== 3. 划分数据（80/20） ====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# =============== Part A: LightGBM 多分类 ===============
from sklearn.metrics import accuracy_score

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# 将标签编码为 0..K-1，以便 LGBM 设置 num_class
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)
num_class = len(le.classes_)

lgbm_clf = lgb.LGBMClassifier(
    boosting_type="gbdt",
    objective="multiclass",
    num_class=num_class,
    max_depth=8,
    num_leaves=31,            # 多分类可适度放宽叶子数
    learning_rate=0.05,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    n_estimators=1500,
    random_state=42
)

lgbm_pipe = Pipeline(steps=[("prep", preprocess), ("model", lgbm_clf)])
lgbm_pipe.fit(X_train, y_train_enc)

proba_lgbm = lgbm_pipe.predict_proba(X_test)  # shape: [n, K]
pred_lgbm  = lgbm_pipe.predict(X_test)

# 多分类 ROC-AUC（宏平均，ovr）
y_test_bin = label_binarize(y_test_enc, classes=np.arange(num_class))
auc_macro_lgbm = roc_auc_score(y_test_bin, proba_lgbm, average="macro", multi_class="ovr")
acc_lgbm = accuracy_score(y_test_enc, pred_lgbm)
print(f"[LGBM-BoxOffice] AUC(macro,ovr)={auc_macro_lgbm:.3f}  Acc={acc_lgbm:.3f}")
print(classification_report(y_test_enc, pred_lgbm, target_names=le.classes_))
print(confusion_matrix(y_test_enc, pred_lgbm))

# =============== Part B: CatBoost 多分类 ===============
# CatBoost 可直接喂入类别列（无需 One-Hot），更贴合“含大量分类变量”的论文设定。:contentReference[oaicite:11]{index=11}
X_train_cb = X_train.copy()
X_test_cb  = X_test.copy()

# 将目标转为 0..K-1
y_train_cb = y_train_enc.copy()
y_test_cb  = y_test_enc.copy()

# 标注类别列索引（CatBoost 需要列索引）
cat_features_idx = [X_train_cb.columns.get_loc(c) for c in cat_cols]

train_pool = Pool(X_train_cb, label=y_train_cb, cat_features=cat_features_idx)
test_pool  = Pool(X_test_cb,  label=y_test_cb,  cat_features=cat_features_idx)

cb = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="TotalF1",   # 也可用 MultiClass/macro-F1
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    iterations=2000,
    random_seed=42,
    verbose=False
)

cb.fit(train_pool, eval_set=test_pool, use_best_model=True)

proba_cb = cb.predict_proba(test_pool)
pred_cb  = cb.predict(test_pool).astype(int).ravel()

auc_macro_cb = roc_auc_score(y_test_bin, proba_cb, average="macro", multi_class="ovr")
acc_cb = accuracy_score(y_test_cb, pred_cb)
print(f"[CatBoost-BoxOffice] AUC(macro,ovr)={auc_macro_cb:.3f}  Acc={acc_cb:.3f}")
print(classification_report(y_test_cb, pred_cb, target_names=le.classes_))
print(confusion_matrix(y_test_cb, pred_cb))


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 763
[LightGBM] [Info] Number of data points in 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LGBM-BoxOffice] AUC(macro,ovr)=0.807  Acc=0.522
              precision    recall  f1-score   support

           1       0.81      0.65      0.72        20
           2       0.48      0.45      0.47        31
           3       0.54      0.59      0.57        44
           4       0.42      0.52      0.46        25
           5       0.00      0.00      0.00         6
           6       0.57      0.50      0.53         8

    accuracy                           0.52       134
   macro avg       0.47      0.45      0.46       134
weighted avg       0.52      0.52      0.52       134

[[13  6  1  0  0  0]
 [ 2 14 11  4  0  0]
 [ 1  8 26  7  1 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha